In [40]:
import pandas as pd
import numpy as np
import urllib
import requests
from bs4 import BeautifulSoup
import os
from time import sleep
import time
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError
import random 

In [41]:
import spacy
nlp = spacy.load("pt_core_news_sm")

In [107]:
df = pd.read_csv("links_estadao_resgatados_vazios.csv")
df.shape


(168, 3)

In [108]:
df.head()

,ano,links,paragrafos_resgatados
0,2001,"https://cultura.estadao.com.br/noticias/geral,...",[]
1,2001,"https://cultura.estadao.com.br/noticias/geral,...",[]
2,2001,https://politica.estadao.com.br/noticias/geral...,[]
3,2002,https://cultura.estadao.com.br/noticias/musica...,[]
4,2002,https://cultura.estadao.com.br/noticias/musica...,[]


In [109]:
def limpa_paragrafos(link, tamanho):
    
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36"}
    try:
        req = Request(link, headers = headers)
        response = urlopen(req)
       
    except HTTPError as e: # erro de segurança do site, pode ser necessário usar user-Agent
        print(e.status, e.reason)
        return ["404_nao_encontrado"]

    except URLError as e:
        print(e.reason)
        return ["404_nao_encontrado"]

    t = random.randint(1, 10)
    # time.sleep(t)
    
#     html = urllib.request.urlopen(link).read()
    html = response.read()
    soup = BeautifulSoup(html)

    for script in soup(["script", "style"]):
        script.decompose()

    lista = []
    for i in soup.stripped_strings:
        lista.append(i)
    
#     print(lista)

    miolo = []
    for i in lista:
        if len(i) > tamanho:
            miolo.append(i.replace('\r\n',''))
#             print(miolo)

    paragrafos = []
    for i in range(len(miolo)):
        if 'esquizo' in miolo[i]:
            paragrafos.append(miolo[i])
    
    t = random.randint(1, 10)
    # time.sleep(t)
    
#     for i in range(len(paragrafos)):
#         lista_final = []
#         doc = nlp(paragrafos[i].lower())
#         for token in doc:
#             valido = not token.is_stop and token.is_alpha
# #             valido = token.is_stop
#             if valido:
#                 lista_final.append(token.text)
    
#         limpo = ' '.join(lista_final)
#         paragrafos[i] = limpo
#         time.sleep(1)

    return paragrafos

In [110]:
link = df.links.values[0]
link

'https://cultura.estadao.com.br/noticias/geral,a-volta-de-jose-agrippino-de-paula-o-bruxo-de-embu,20010406p3412'

In [111]:
paragrafos = limpa_paragrafos(link, 120)

In [112]:
str(paragrafos)

'[]'

In [91]:
df.links.iloc[1]

'https://economia.estadao.com.br/noticias/geral,pf-prende-17-pessoas-por-fraude-de-r-11-milhoes-ao-inss,425045'

In [116]:
n_links = df.shape[0]

In [117]:
for i in range(0, n_links):
    link = df.links.iloc[i]
    paragrafos = limpa_paragrafos(link, 70)
    df.loc[i, "paragrafos_resgatados"] = str(paragrafos)

In [118]:
df

,ano,links,paragrafos_resgatados
0,2001,"https://cultura.estadao.com.br/noticias/geral,...",[]
1,2001,"https://cultura.estadao.com.br/noticias/geral,...",[]
2,2001,https://politica.estadao.com.br/noticias/geral...,[]
3,2002,https://cultura.estadao.com.br/noticias/musica...,[]
4,2002,https://cultura.estadao.com.br/noticias/musica...,[]
...,...,...,...
163,2022,"https://emais.estadao.com.br/fotos/bem-estar,d...",['Dicionário anticapacitista pretende combater...
164,2022,"https://emais.estadao.com.br/noticias/gente,fi...",[]
165,2022,https://emais.estadao.com.br/noticias/bem-esta...,[]
166,2022,"https://saude.estadao.com.br/noticias/geral,am...",[]


In [121]:
df2 = pd.DataFrame(columns = ["ano", "links", "paragrafos_resgatados"])
for i in range(1, 5):
    df1 = pd.read_csv(f"links_estadao_resgatados_parte_{i}.csv")
    df2 = pd.concat([df2, df1])


In [122]:
df2.to_csv("links_estadao_resgatados.csv", index = False)

In [102]:
# df[df.paragrafos_resgatados.notnull()].to_csv("links_estadao_resgatados_parte_3.csv", index = False)
# df[df.paragrafos_resgatados.isnull()].to_csv("links_estadao_nulos.csv", index = False)


In [106]:
# df = pd.read_csv("links_estadao_resgatados.csv")
# df.shape

In [119]:
t = df.query("paragrafos_resgatados == '[]'")
t

,ano,links,paragrafos_resgatados
0,2001,"https://cultura.estadao.com.br/noticias/geral,...",[]
1,2001,"https://cultura.estadao.com.br/noticias/geral,...",[]
2,2001,https://politica.estadao.com.br/noticias/geral...,[]
3,2002,https://cultura.estadao.com.br/noticias/musica...,[]
4,2002,https://cultura.estadao.com.br/noticias/musica...,[]
...,...,...,...
157,2021,https://politica.estadao.com.br/blogs/coluna-d...,[]
164,2022,"https://emais.estadao.com.br/noticias/gente,fi...",[]
165,2022,https://emais.estadao.com.br/noticias/bem-esta...,[]
166,2022,"https://saude.estadao.com.br/noticias/geral,am...",[]


In [34]:
t.links.values 

array(['https://www.correiodopovo.com.br/not%C3%ADcias/pol%C3%ADcia/restos-mortais-de-v%C3%ADtimas-de-canibalismo-s%C3%A3o-enterrados-em-pe-1.89468',
       'https://www.correiodopovo.com.br/not%C3%ADcias/geral/anvisa-suspende-com%C3%A9rcio-de-lotes-de-rem%C3%A9dio-para-esquizofrenia-1.140120',
       'https://www.correiodopovo.com.br/arteagenda/peter-green-ex-guitarrista-do-fleetwood-mac-morre-aos-73-anos-1.455521',
       'https://www.correiodopovo.com.br/blogs/juremirmachado/dilma-furiosa-na-onu-e-psb-fervendo-no-rs-1.550',
       'https://www.correiodopovo.com.br/blogs/hiltormombach/eles-vivem-em-outro-planeta-1.409755',
       'https://www.correiodopovo.com.br/blogs/juremirmachado/votos-que-valem-mais-1.287780'],
      dtype=object)

In [123]:
t.to_csv("links_estadao_resgatados_vazios.csv", index = False)

In [ ]:
# os links que não funcionaram são apenas TÍTULOS, ou trechos que contém menos de 120 caracteres

In [36]:
df["indice"] = [x for x in range(df.shape[0])]